In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import onnxruntime
import tf2onnx

print("TensorFlow version:", tf.__version__)

In [ ]:
from tensorflow.python.client import device_lib
device_name = [x.name for x in device_lib.list_local_devices() if x.device_type == 'GPU']

## Load Data

The data can be found in CSV files `Furnace/data/eq_*.csv`.

In [ ]:
eq_1215_df = pd.read_csv("Furnace/data/eq_1215.csv")
eq_1215_n_inputs = 14
eq_1215_n_outputs = 18
eq_1215_n_samples = len(eq_1215_df)

assert eq_1215_n_inputs+eq_1215_n_outputs == len(eq_1215_df.columns), "Sizes don't match"

eq_1215_input_names = eq_1215_df.columns[:eq_1215_n_inputs]
eq_1215_output_names = eq_1215_df.columns[-eq_1215_n_outputs:]

eq_1215_df.head()

In [ ]:
eq_1215_train_inputs = np.array(eq_1215_df[eq_1215_input_names])
eq_1215_train_outputs = np.array(eq_1215_df[eq_1215_output_names])

display(eq_1215_train_inputs)

## Model

In Julia we used a model like:

```julia
  model = Flux.Chain(Flux.Dense(nInputs,     nInputs*10,  Flux.σ),
                     Flux.Dense(nInputs*10,  nOutputs*10, tanh),
                     Flux.Dense(nOutputs*10, nOutputs))
```


In [ ]:
eq_1215_model = tf.keras.models.Sequential([
  layers.Dense(eq_1215_n_inputs, activation='sigmoid'),
  layers.Dense(eq_1215_n_inputs*10, activation='tanh'),
  layers.Dense(eq_1215_n_outputs)
])

In [ ]:
eq_1215_model.compile(loss = tf.keras.losses.MeanSquaredError(),
                      optimizer = tf.keras.optimizers.Adam())

## Training



In [ ]:
eq_1215_model.fit(eq_1215_train_inputs, eq_1215_train_outputs, epochs=10)
#eq_1215_model.save(os.path.join("onnx", "eq_1215"))


## Export to ONNX

Use `onnxruntime-gpu` and `tf2onnx` to export model to ONNX.

In [ ]:
spec = (tf.TensorSpec((None, eq_1215_n_inputs,), tf.float32, name="input"),)
output_path = os.path.join("Furnace", "onnx", "eq_1215.onnx")

model_proto, _ = tf2onnx.convert.from_keras(eq_1215_model, input_signature=spec, opset=12, output_path=output_path)

## Do the rest

In [ ]:
def getNInput(eq):
  if eq == "1215":
    return 14
  elif eq == "1585":
    return 20
  elif eq == "1933":
    return 20
  else:
    raise(IndexError('Not allowed eq value ' + eq))

for eq in ["1585", "1933"]:
  eq_df = pd.read_csv("Furnace/data/eq_"+eq+".csv")
  eq_n_inputs = getNInput(eq)
  eq_n_outputs = len(eq_df.columns)-eq_n_inputs

  eq_input_names = eq_df.columns[:eq_n_inputs]
  eq_output_names = eq_df.columns[eq_n_inputs:]

  eq_train_inputs = np.array(eq_df[eq_input_names])
  eq_train_outputs = np.array(eq_df[eq_output_names])

  model = tf.keras.models.Sequential([
    layers.Dense(eq_n_inputs, activation='sigmoid'),
    layers.Dense(eq_n_inputs*10, activation='tanh'),
    layers.Dense(eq_n_outputs)
  ])
  model.compile(loss = tf.keras.losses.MeanSquaredError(),
                optimizer = tf.keras.optimizers.Adam())
  model.fit(eq_train_inputs, eq_train_outputs, epochs=10)
  #model.save(os.path.join("onnx", "eq_"+eq))

  spec = (tf.TensorSpec((None, eq_n_inputs,), tf.float32, name="input"),)
  output_path = os.path.join("Furnace", "onnx", "eq_"+eq+".onnx")
  model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=12, output_path=output_path)

